# Calling MATLAB from Python
In this Jupyter notebook, I will briefly go over how to call MATLAB from Python. 
To learn how to call MATLAB from Python, check this [live script](https://github.com/DeepakGuru1998/Matlab-with-Python/blob/main/pythonFromMatlab_demo.mlx).

To access MATLAB from Python, you need MATLAB engine API for Python, which you can install as per the instructions given [here](https://www.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html).

This tutorial assumes that you already know how to use basic Python and MATLAB, and that you have ran `python setup.py install`

**MATLAB API for Python**

To the MATLAB®  Engine API for Python® you will need to have a copy of MATLAB installed in you system. There is no workaround for this, as far as I know, and this is a consequence of MATLAB being a proprietary software. This API supports almost every version of Python, and requires **CPython** to be installed on your system, in order to use the referencing of inputs and outputs required to exchange arguments between the two worlds.

## Getting started with MATLAB in Python
The first step is to import the MATLAB module (and some other packages which are optional) as follows:

In [1]:
import matlab.engine
import numpy as np

In [2]:
from numpy import *
from pylab import *

In [ ]:
cd "matlabroot/extern/engines/python"

In [ ]:
!pip python setup.py install

where `matlabroot` is the path where you installed MATLAB on your system.

That’s it!

The API provides a Python package named matlab that enables you to call MATLAB functions from Python.

Start a MATLAB session:

In [3]:
eng = matlab.engine.start_matlab()

# Call MATLAB (built-in) functions from Python

The usage of MATLAB in Python is similar to the syntax of Python in MATLAB. We can call any MATLAB command using `eng.[command]`. 



You can call any MATLAB function directly and return the results to Python. This holds as long as the function can be found in MATLAB’s path (we will come beck to this shortly).

For example, to determine if a number is prime, use the engine to call the isprime function.

In [4]:
tf = eng.isprime(37)
print(tf)
print(type(tf))

True
<class 'bool'>


This was a simple one: the MATALB function we call produced only one output, and it was a ‘scalar’ (actually boolean) output, not an array of some type.

## Transfering variables from Python to MATLAB workspace

When you start the engine, it provides an interface to a collection of all MATLAB variables. This collection, named workspace, is implemented as a Python dictionary that is attached to the engine:

  >The name of each MATLAB variable becomes a key in the workspace dictionary.
  
  >The keys in workspace must be valid MATLAB identifiers (e.g., you cannot use numbers as keys).
  
  
You can add variables to the engine workspace in Python, and then you can use the variables in MATLAB functions:

In [5]:
# variable x in Python workspace
x = 4.0
# a new variable called y is added to MATLAB workspace, and is value is set to be equal to Python's x
eng.workspace['y'] = x
# we can use variable y while calling MATLAB functions, ad MATLAB is aware of all the variable availabe in its workspace
a = eng.eval('sqrt(y)')
print(a)

2.0


In this example, x exists only as a Python variable. Its value is assigned to a new entry in the engine workspace, called y, creating a MATLAB variable. You can then call the MATLAB `eval` function to execute the `sqrt(y)` statement in MATLAB and return the output value, 2.0, to Python.

## Use MATLAB Arrays in Python


Usually, while working with MATLAB, we are interested in performing complex operations on arrays. The `matlab` package provides constructors to create MATLAB arrays in Python. The MATLAB Engine API for Python can pass such arrays as input arguments to MATLAB functions, and can return such arrays as output arguments to Python.

In [7]:
#You can create arrays of any MATLAB numeric or logical type from Python sequence types, as follows:
a = matlab.double([1,4,9,16,25])
b = eng.sqrt(a)
print(b)
print(type(b))

#The engine returns b, which is a 1-by-5 matlab.double array.

[[1.0,2.0,3.0,4.0,5.0]]
<class 'mlarray.double'>


In [8]:
#The same applies if we want to create a multidimensional array. 
#The magic function returns a 2-D matlab.double array to Python.
a = eng.magic(6)
for x in a: 
    print(x)
print(type(a))


[35.0,1.0,6.0,26.0,19.0,24.0]
[3.0,32.0,7.0,21.0,23.0,25.0]
[31.0,9.0,2.0,22.0,27.0,20.0]
[8.0,28.0,33.0,17.0,10.0,15.0]
[30.0,5.0,34.0,12.0,14.0,16.0]
[4.0,36.0,29.0,13.0,18.0,11.0]
<class 'mlarray.double'>


Unfortunately, `matlab` package seems to work only with pure Python data structures, meaning that we will need to use some tricks if we are interested in working with, e.g., `numpy arrays`. This is important, as usually if we need to call a MATALB function to work on arrays, it is because in Python we were working with arrays and this is usually done via `numpy`.

In [10]:
import numpy as np

a = np.array([1,2,3,4]).reshape([1,4])
print(a)
b = a**2
print(type(a))
print(type(b))
print(b)
print(b.shape)

[[1 2 3 4]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[[ 1  4  9 16]]
(1, 4)


We created a numpy array a, and then we compute the square of each of its values, yelding another numpy array.

If we try to reproduce this operation using matlab package we will be stuck in an error as soon as we try to cast the numpy array a as a matlab.double array:

In [11]:
a_m = matlab.double(a)


ValueError: initializer must be a rectangular nested sequence

This happens because `matlab.double` function is expecting a `list` or a `tuple` as input, and it is unable to understand the `numpy.ndarray` datatype.

A workaraound is to go back to the list format:

In [12]:
a_m = matlab.double(a.tolist()) # casting a as list
b_m = eng.power(a_m,2.0)
print((b_m))
print(type(b_m))
print(b_m.size)

[[1.0,4.0,9.0,16.0]]
<class 'mlarray.double'>
(1, 4)


End the MATLAB session:

In [9]:
eng.quit()

Read the parquet file 
<br> **Requirements:** Pyarrow or fastparquet or snappy engine to read the data (Use pip install or conda install) 


In [10]:
import pandas as pd
import pyarrow #compression technique

In [11]:
data=pd.read_parquet('sample.parquet')
data

,Var1,Var2,Var3,Var4,Var5,Var6,Var7,Var8,Var9,Var10
0,1.0,"Eldon Base for stackable storage shelf, platinum",Muhammed MacIntyre,3.0,-213.25,38.94,35.00,Nunavut,Storage & Organization,0.80
1,2.0,"1.7 Cubic Foot Compact ""Cube"" Office Refrigera...",Barry French,293.0,457.81,208.16,68.02,Nunavut,Appliances,0.58
2,3.0,"Cardinal Slant-D® Ring Binder, Heavy Gauge Vinyl",Barry French,293.0,46.71,8.69,2.99,Nunavut,Binders and Binder Accessories,0.39
3,4.0,R380,Clay Rozendal,483.0,1198.97,195.99,3.99,Nunavut,Telephones and Communication,0.58
4,5.0,Holmes HEPA Air Purifier,Carlos Soltero,515.0,30.94,21.78,5.94,Nunavut,Appliances,0.50
...,...,...,...,...,...,...,...,...,...,...
9995,9996.0,"Eaton Premium Continuous-Feed Paper, 25% Cotto...",Corey Catlett,43815.0,539.45,55.48,6.79,British Columbia,Paper,0.37
9996,9997.0,Space Solutions Industrial Galvanized Steel S...,Corey Catlett,43815.0,-1097.97,78.80,35.00,British Columbia,Storage & Organization,0.83
9997,9998.0,KF 788,Deborah Brumfield,43844.0,170.57,45.99,4.99,British Columbia,Telephones and Communication,0.56
9998,9999.0,Telephone Message Books with Fax/Mobile Sectio...,Jill Matthias,43846.0,-8.28,3.60,2.20,British Columbia,Paper,0.39
